# 🧪 Clinical Laboratory Inventory Diagnostic & Testing System

## 🎯 Purpose
This notebook is designed to **diagnose and test** the hospital inventory system's location-based query functionality. We'll verify that the AI agent can correctly query the database and return actual inventory items instead of generic responses.

## 🔍 What We're Testing
1. **Database Connectivity** - Verify connection to PostgreSQL database
2. **Location-Based Queries** - Test filtering by Clinical Laboratory location
3. **Action Execution** - Debug why the web interface shows generic responses
4. **Response Generation** - Analyze how the AI agent processes inventory data

## 🚨 Current Issue
The system returns generic responses like:
> "Based on the inventory inquiry for the Clinical Laboratory location, here are the items typically found..."

Instead of actual database items like:
> "Blood Collection Tubes (100 pack), Reagent Test Kits, Microscope Slides..."

In [1]:
# Import Required Libraries for Diagnostic Testing
import sys
import os
import asyncio
import json
import pandas as pd
from datetime import datetime
from pathlib import Path

# Add current directory to Python path for imports
current_dir = Path().absolute()
sys.path.insert(0, str(current_dir))
sys.path.insert(0, str(current_dir.parent / "backend"))

print("📦 Imported core libraries")
print(f"📁 Working directory: {current_dir}")
print(f"🐍 Python path includes: {current_dir} and {current_dir.parent / 'backend'}")

# Import our custom modules
try:
    from comprehensive_ai_agent import ComprehensiveAIAgent
    print("✅ Successfully imported ComprehensiveAIAgent")
except ImportError as e:
    print(f"❌ Failed to import ComprehensiveAIAgent: {e}")

try:
    from fixed_database_integration import FixedDatabaseIntegration
    print("✅ Successfully imported FixedDatabaseIntegration") 
except ImportError as e:
    print(f"❌ Failed to import FixedDatabaseIntegration: {e}")

print("\n🔧 Import Setup Complete!")

📦 Imported core libraries
📁 Working directory: e:\Rise Ai\Updated\Hospital-Operations-Logistics-Agentic-Platform_\ai_ml
🐍 Python path includes: e:\Rise Ai\Updated\Hospital-Operations-Logistics-Agentic-Platform_\ai_ml and e:\Rise Ai\Updated\Hospital-Operations-Logistics-Agentic-Platform_\backend


c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Successfully imported ComprehensiveAIAgent
✅ Successfully imported FixedDatabaseIntegration

🔧 Import Setup Complete!


In [5]:
# Test Database Connectivity
async def test_database_connection():
    """Test direct database connectivity and data retrieval"""
    print("🔌 Testing Database Connection...")
    
    try:
        # Initialize database integration
        db = FixedDatabaseIntegration()
        
        # Initialize database connection (correct method)
        connection_success = await db.initialize()
        if connection_success:
            print("✅ Database connection successful!")
        else:
            print("❌ Database initialization failed!")
            return False, 0, 0
        
        # Test basic inventory query
        inventory_data = await db.get_inventory_data()
        items = inventory_data.get('items', [])
        
        print(f"📊 Found {len(items)} total inventory items in database")
        
        # Show sample of items
        if items:
            print("\n📦 Sample inventory items:")
            for i, item in enumerate(items[:5]):
                print(f"  {i+1}. {item.get('item_name', item.get('name', 'Unknown'))} - Location: {item.get('location_name', item.get('location_id', 'Unknown'))}")
        
        # Test location-specific query
        print("\n🧪 Testing Clinical Laboratory location query...")
        lab_inventory = await db.get_inventory_by_location(location_name="Clinical Laboratory")
        lab_items = lab_inventory.get('items', [])
        
        print(f"📋 Found {len(lab_items)} items in Clinical Laboratory location")
        
        if lab_items:
            print("\n🔬 Clinical Laboratory inventory:")
            for i, item in enumerate(lab_items):
                print(f"  {i+1}. {item.get('item_name', 'Unknown')} - Stock: {item.get('current_stock', 0)} {item.get('unit_of_measure', 'units')}")
        else:
            print("⚠️ No items found for Clinical Laboratory - trying alternative location names...")
            
            # Try different location variations
            alternative_names = ["clinical laboratory", "lab", "LAB-01", "Clinical Lab"]
            for alt_name in alternative_names:
                print(f"   🔍 Trying '{alt_name}'...")
                lab_inventory = await db.get_inventory_by_location(location_name=alt_name) 
                lab_items = lab_inventory.get('items', [])
                if lab_items:
                    print(f"   ✅ Found {len(lab_items)} items with location name '{alt_name}'")
                    break
            
            if not lab_items:
                print("   ⚠️ No items found with any location variation")
                
                # Show available locations for debugging
                print("\n📍 Checking available locations...")
                try:
                    # Custom query to get all locations since get_all_locations doesn't exist
                    from sqlalchemy import text
                    async with db.async_session() as session:
                        locations_result = await session.execute(text("""
                            SELECT location_id, name, location_type
                            FROM locations WHERE is_active = TRUE
                            ORDER BY name
                        """))
                        locations = [
                            {
                                "location_id": row[0],
                                "location_name": row[1],
                                "location_type": row[2]
                            }
                            for row in locations_result.fetchall()
                        ]
                    
                    if locations:
                        print(f"   Available locations ({len(locations)}):")
                        for loc in locations[:15]:  # Show first 15
                            print(f"     - {loc.get('location_name', 'Unknown')} (ID: {loc.get('location_id', 'Unknown')})")
                        
                        if len(locations) > 15:
                            print(f"     ... and {len(locations) - 15} more locations")
                    else:
                        print("   ❌ No locations found in database")
                        
                except Exception as e:
                    print(f"   ❌ Could not fetch locations: {e}")
            
        return True, len(items), len(lab_items)
        
    except Exception as e:
        print(f"❌ Database test failed: {e}")
        import traceback
        print(f"🔍 Error details:\n{traceback.format_exc()}")
        return False, 0, 0

# Run the database test
db_success, total_items, lab_items = await test_database_connection()
print(f"\n📊 Database Test Results:")
print(f"   - Connection: {'✅ SUCCESS' if db_success else '❌ FAILED'}")
print(f"   - Total Items: {total_items}")
print(f"   - Lab Items: {lab_items}")

🔌 Testing Database Connection...
✅ Database connection successful!
📊 Found 124 total inventory items in database

📦 Sample inventory items:
  1. Surgical Gloves (Box of 100) - Location: ICU-01
  2. Surgical Gloves (Box of 100) - Location: SURGERY-01
  3. Surgical Gloves (Box of 100) - Location: WAREHOUSE
  4. N95 Respirator Masks (20 pack) - Location: ER-01
  5. N95 Respirator Masks (20 pack) - Location: ICU-01

🧪 Testing Clinical Laboratory location query...
📋 Found 4 items in Clinical Laboratory location

🔬 Clinical Laboratory inventory:
  1. Blood Collection Tubes (100 pack) - Stock: 34 pack
  2. Disinfectant Wipes (Pack of 100) - Stock: 39 pack
  3. Hand Sanitizer 500ml - Stock: 38 bottle
  4. Hospital-Grade Disinfectant 1L - Stock: 16 bottle

📊 Database Test Results:
   - Connection: ✅ SUCCESS
   - Total Items: 124
   - Lab Items: 4


In [6]:
# Test AI Agent Location-Based Inventory Query
async def test_ai_agent_query():
    """Test the AI agent's location-based inventory functionality"""
    print("🤖 Testing AI Agent Location Query...")
    
    try:
        # Initialize the AI agent
        agent = ComprehensiveAIAgent()
        print("✅ AI Agent initialized successfully")
        
        # Test the problematic query
        test_query = "what are the inventory items I have in Clinical Laboratory location?"
        print(f"\n📝 Testing query: '{test_query}'")
        
        # Process the conversation
        response = await agent.process_conversation(test_query)
        
        print("\n📊 Agent Response Analysis:")
        print(f"   - Session ID: {response.get('session_id', 'None')}")
        print(f"   - Intent: {response.get('intent_analysis', {}).get('primary_intent', 'Unknown')}")
        print(f"   - Actions Count: {len(response.get('actions_executed', []))}")
        
        # Analyze actions executed
        actions = response.get('actions_executed', [])
        if actions:
            print("\n🔧 Actions Executed:")
            for i, action in enumerate(actions):
                print(f"   {i+1}. Type: {action.get('action_type', 'Unknown')}")
                print(f"      Description: {action.get('description', 'No description')}")
                print(f"      Parameters: {action.get('parameters', {})}")
        else:
            print("⚠️ No actions were executed!")
        
        # Show the actual response
        actual_response = response.get('response', 'No response')
        print(f"\n💬 Actual Response:")
        print("=" * 80)
        print(actual_response)
        print("=" * 80)
        
        # Check if response contains generic keywords (indicating problem)
        generic_keywords = [
            "typically found", 
            "items such as", 
            "generally includes",
            "commonly stored",
            "usually contains"
        ]
        
        is_generic = any(keyword in actual_response.lower() for keyword in generic_keywords)
        
        print(f"\n🔍 Response Analysis:")
        print(f"   - Contains generic keywords: {'❌ YES (PROBLEM)' if is_generic else '✅ NO (GOOD)'}")
        print(f"   - Response length: {len(actual_response)} characters")
        print(f"   - Mentions database items: {'✅ YES' if any(word in actual_response.lower() for word in ['blood', 'reagent', 'pipette']) else '❌ NO'}")
        
        return response, is_generic
        
    except Exception as e:
        print(f"❌ AI Agent test failed: {e}")
        import traceback
        traceback.print_exc()
        return None, True

# Run the AI agent test
agent_response, is_generic_response = await test_ai_agent_query()

print(f"\n🎯 AI Agent Test Results:")
print(f"   - Agent Response: {'❌ GENERIC' if is_generic_response else '✅ SPECIFIC'}")
print(f"   - Issue Status: {'🚨 NEEDS FIXING' if is_generic_response else '✅ WORKING CORRECTLY'}")

🤖 Testing AI Agent Location Query...
✅ AI Agent initialized successfully

📝 Testing query: 'what are the inventory items I have in Clinical Laboratory location?'

📊 Agent Response Analysis:
   - Session ID: 7f32b646-a149-48ea-a457-b3de0f584fac
   - Intent: Unknown
   - Actions Count: 0
⚠️ No actions were executed!

💬 Actual Response:
In the Clinical Laboratory location, you currently have the following inventory items available:
1. Item ID: ITEM-009
Item Name: Blood Collection Tubes (100 pack)
Description: Essential for blood sample collection and storage.
For a detailed inventory list or if you require additional information on specific items, feel free to provide more details for a comprehensive inventory overview.


**Actions Performed:**
✅ Get inventory items for location: laboratory

🔍 Response Analysis:
   - Contains generic keywords: ✅ NO (GOOD)
   - Response length: 462 characters
   - Mentions database items: ✅ YES

🎯 AI Agent Test Results:
   - Agent Response: ✅ SPECIFIC
   -

In [11]:
# Debug Action Execution Step-by-Step
async def debug_action_execution():
    """Debug each step of the action execution process"""
    print("🔍 Debugging Action Execution Process...")
    
    try:
        agent = ComprehensiveAIAgent()
        test_message = "what are the inventory items I have in Clinical Laboratory location?"
        
        print(f"📝 Testing message: '{test_message}'")
        
        # Step 1: Test intent analysis
        # Import required classes from the agent module
        from comprehensive_ai_agent import ConversationMemory, ConversationContext
        from datetime import datetime
        
        # Check if we have an existing memory or create a new one
        session_id = "debug_session"
        if session_id in agent.conversation_memory:
            memory = agent.conversation_memory[session_id]
            print(f"   🔄 Using existing memory for session: {session_id}")
        else:
            # Create a properly initialized ConversationMemory
            memory = ConversationMemory(
                user_id="debug_user",
                session_id=session_id,
                context_type=ConversationContext.INVENTORY_INQUIRY,
                entities_mentioned=[],
                actions_performed=[],
                preferences={},
                last_updated=datetime.now()
            )
            agent.conversation_memory[session_id] = memory
            print(f"   ✨ Created new memory for session: {session_id}")
        
        print(f"\n🧠 Step 1 - Memory Setup:")
        print(f"   - Memory object: {type(memory).__name__}")
        print(f"   - Context type: {memory.context_type}")
        print(f"   - Actions performed: {len(memory.actions_performed)}")
        print(f"   - Conversation history: {len(memory.conversation_history)}")
        
        # Test if the _analyze_user_intent method exists
        if hasattr(agent, '_analyze_user_intent'):
            print(f"\n🎯 Step 2 - Intent Analysis:")
            try:
                intent_analysis = await agent._analyze_user_intent(test_message, memory)
                
                print(f"   - Analysis completed successfully!")
                print(f"   - Primary Intent: {intent_analysis.get('primary_intent', 'Not found')}")
                print(f"   - Primary Intent String: {intent_analysis.get('primary_intent_str', 'Not found')}")
                print(f"   - Entities Found: {intent_analysis.get('entities', [])}")
                print(f"   - Urgency: {intent_analysis.get('message_analysis', {}).get('urgency_level', 'Unknown')}")
                
                # Check specific intent for inventory queries
                if hasattr(intent_analysis.get('primary_intent'), 'value'):
                    intent_value = intent_analysis['primary_intent'].value
                    print(f"   - Intent Value: {intent_value}")
                    if intent_value == "inventory_inquiry":
                        print(f"   ✅ Correctly identified as inventory inquiry!")
                    else:
                        print(f"   ⚠️ Intent not identified as inventory inquiry")
                
            except Exception as e:
                print(f"   ❌ Intent analysis failed: {e}")
                intent_analysis = {}
        else:
            print(f"\n❌ Step 2 - Intent Analysis method not found!")
            intent_analysis = {}
        
        # Step 3: Test action determination
        if hasattr(agent, '_determine_required_actions') and intent_analysis:
            print(f"\n⚙️ Step 3 - Action Determination:")
            try:
                required_actions = await agent._determine_required_actions(test_message, intent_analysis, memory)
                
                print(f"   - Actions determined: {len(required_actions)}")
                for i, action in enumerate(required_actions):
                    print(f"   - Action {i+1}: {action.action_type}")
                    print(f"     Description: {action.description}")
                    print(f"     Parameters: {action.parameters}")
                    
                    # Check for location-based action
                    if "location" in action.action_type.lower() or any("location" in str(param).lower() for param in action.parameters.values()):
                        print(f"     ✅ Location-based action detected!")
                        
            except Exception as e:
                print(f"   ❌ Action determination failed: {e}")
                required_actions = []
        else:
            print(f"\n❌ Step 3 - Action determination not available")
            required_actions = []
        
        # Step 4: Test individual action execution
        if required_actions and hasattr(agent, '_execute_action'):
            print(f"\n🔧 Step 4 - Action Execution:")
            for i, action in enumerate(required_actions):
                try:
                    result = await agent._execute_action(action, session_id)
                    print(f"   - Action {i+1} Result: {type(result)} with {len(str(result))} chars")
                    
                    if isinstance(result, dict):
                        if "error" in result:
                            print(f"     ❌ Error: {result['error']}")
                        elif "items" in result:
                            items = result["items"]
                            print(f"     ✅ Success: Found {len(items)} items")
                            if items:
                                print(f"     📦 Sample item: {items[0].get('item_name', 'Unknown')}")
                        else:
                            print(f"     📊 Result keys: {list(result.keys())}")
                    
                except Exception as e:
                    print(f"   - Action {i+1} Failed: {e}")
        
        elif required_actions:
            print(f"\n❌ Step 4 - Action execution method not found!")
        else:
            print("⚠️ No actions were determined to execute!")
        
        # Step 5: Test direct process_conversation to compare
        print(f"\n🔄 Step 5 - Direct Message Processing:")
        try:
            # Use the correct method name: process_conversation
            direct_result = await agent.process_conversation(
                user_message=test_message,
                user_id="debug_user",
                session_id=session_id
            )
            
            # Extract the response from the result dictionary
            if isinstance(direct_result, dict) and "response" in direct_result:
                direct_response = direct_result["response"]
                print(f"   ✅ Direct processing successful!")
                print(f"   - Response type: {type(direct_response)}")
                print(f"   - Response length: {len(str(direct_response))} characters")
                print(f"   - Actions executed: {len(direct_result.get('actions', []))}")
                
                # Handle response display
                response_str = str(direct_response)
                if len(response_str) > 200:
                    print(f"   - Response preview: {response_str[:200]}...")
                else:
                    print(f"   - Full response: {response_str}")
                
                # Check if response is specific or generic
                specific_indicators = ["Blood Collection Tubes", "ITEM-009", "34 pack", "Clinical Laboratory"]
                generic_indicators = ["typically found", "generally includes", "commonly contains"]
                
                contains_specific = any(indicator in response_str for indicator in specific_indicators)
                contains_generic = any(indicator in response_str for indicator in generic_indicators)
                
                if contains_specific:
                    print(f"   ✅ Response contains specific data")
                elif contains_generic:
                    print(f"   ❌ Response contains generic language")
                else:
                    print(f"   ⚠️ Response type unclear")
                    
                # Show the actions that were performed
                if "actions" in direct_result and direct_result["actions"]:
                    print(f"   📋 Actions performed:")
                    for i, action in enumerate(direct_result["actions"]):
                        print(f"     {i+1}. {action.get('action_type', 'Unknown')} - {action.get('description', 'No description')}")
                        
            else:
                print(f"   ❌ Unexpected result format: {type(direct_result)}")
                print(f"   📊 Result keys: {list(direct_result.keys()) if isinstance(direct_result, dict) else 'Not a dict'}")
                
        except Exception as e:
            print(f"   ❌ Direct processing failed: {e}")
            import traceback
            print(f"     Error details: {traceback.format_exc()}")
            
        return intent_analysis, required_actions
        
    except Exception as e:
        print(f"❌ Debug failed: {e}")
        import traceback
        traceback.print_exc()
        return None, []

# Run the debug analysis
debug_intent, debug_actions = await debug_action_execution()

print(f"\n🔬 Debug Analysis Complete:")
print(f"   - Intent Analysis: {'✅ SUCCESS' if debug_intent else '❌ FAILED'}")
print(f"   - Actions Generated: {len(debug_actions) if debug_actions else 0}")
print(f"   - Root Cause: {'🎯 IDENTIFIED' if debug_actions else '🔍 INVESTIGATING'}")

🔍 Debugging Action Execution Process...
📝 Testing message: 'what are the inventory items I have in Clinical Laboratory location?'
   ✨ Created new memory for session: debug_session

🧠 Step 1 - Memory Setup:
   - Memory object: ConversationMemory
   - Context type: ConversationContext.INVENTORY_INQUIRY
   - Actions performed: 0
   - Conversation history: 0

🎯 Step 2 - Intent Analysis:
   - Analysis completed successfully!
   - Primary Intent: ConversationContext.INVENTORY_INQUIRY
   - Primary Intent String: inventory_inquiry
   - Entities Found: ['items', 'laboratory', 'Clinical']
   - Urgency: normal
   - Intent Value: inventory_inquiry
   ✅ Correctly identified as inventory inquiry!

⚙️ Step 3 - Action Determination:
   - Actions determined: 1
   - Action 1: get_inventory_by_location
     Description: Get inventory items for location: laboratory
     Parameters: {'location_name': 'laboratory', 'include_details': True, 'include_summary': True}
     ✅ Location-based action detected!

🔧 

In [23]:
# Comprehensive System Validation
async def comprehensive_validation():
    """Complete end-to-end validation test"""
    print("🏥 COMPREHENSIVE HOSPITAL INVENTORY SYSTEM VALIDATION")
    print("=" * 60)
    
    # Initialize components
    agent = ComprehensiveAIAgent()
    db_integration = FixedDatabaseIntegration()
    
    validation_results = {
        "database_connection": False,
        "location_filtering": False,
        "ai_agent_response": False,
        "action_execution": False,
        "response_specificity": False
    }
    
    # Test 1: Database Connection
    print("\n🔗 Test 1: Database Connection")
    try:
        # Initialize the database connection first
        connection_success = await db_integration.initialize()
        if connection_success:
            print(f"   ✅ Database initialized successfully")
            validation_results["database_connection"] = True
            
            # Test getting inventory data
            inventory_data = await db_integration.get_inventory_data()
            items = inventory_data.get('items', [])
            print(f"   📊 Found {len(items)} total inventory items")
        else:
            print("   ❌ Database initialization failed")
    except Exception as e:
        print(f"   ❌ Database connection error: {e}")
    
    # Test 2: Location-Based Filtering
    print("\n📍 Test 2: Location-Based Inventory Filtering")
    try:
        clinical_items_result = await db_integration.get_inventory_by_location("Clinical Laboratory")
        clinical_items = clinical_items_result.get('items', [])
        
        if clinical_items and len(clinical_items) > 0:
            validation_results["location_filtering"] = True
            print(f"   ✅ Location filtering works - {len(clinical_items)} items in Clinical Laboratory")
            print(f"   📦 Sample items: {[item['item_name'] for item in clinical_items[:3]]}")
        else:
            print("   ❌ Location filtering failed - no items found")
    except Exception as e:
        print(f"   ❌ Location filtering error: {e}")
    
    # Test 3: AI Agent Response
    print("\n🤖 Test 3: AI Agent Response Generation")
    test_queries = [
        "what are the inventory items I have in Clinical Laboratory location?",
        "show me items in the Clinical Laboratory",
        "list inventory for Clinical Laboratory location"
    ]
    
    for i, query in enumerate(test_queries, 1):
        try:
            print(f"\n   Query {i}: '{query}'")
            
            # Use the correct method name: process_conversation
            response_result = await agent.process_conversation(
                user_message=query, 
                user_id="validation_test", 
                session_id=f"validation_{i}"
            )
            
            # Extract response from the result dictionary
            if isinstance(response_result, dict) and "response" in response_result:
                response = response_result["response"]
                
                if response and len(response) > 50:
                    validation_results["ai_agent_response"] = True
                    
                    # Check if response contains specific item names
                    specific_indicators = ["Blood Collection Tubes", "Disinfectant Wipes", "Hand Sanitizer", 
                                         "Hospital-Grade Disinfectant", "ITEM-009", "34 pack"]
                    
                    contains_specific = any(indicator.lower() in response.lower() for indicator in specific_indicators)
                    generic_indicators = ["typically found", "generally includes", "commonly contains", "usually has"]
                    contains_generic = any(indicator.lower() in response.lower() for indicator in generic_indicators)
                    
                    if contains_specific and not contains_generic:
                        validation_results["response_specificity"] = True
                        print(f"   ✅ Response is specific and database-driven")
                    elif contains_generic:
                        print(f"   ❌ Response contains generic language")
                    else:
                        print(f"   ⚠️ Response unclear - manual review needed")
                    
                    print(f"   📝 Response length: {len(response)} characters")
                    print(f"   🎯 First 200 chars: {response[:200]}...")
                    
                    # Show actions executed
                    actions = response_result.get("actions", [])
                    if actions:
                        print(f"   ⚙️ Actions executed: {len(actions)}")
                        for action in actions:
                            print(f"     - {action.get('action_type', 'Unknown')}")
                    
                else:
                    print(f"   ❌ AI Agent response too short or empty")
            else:
                print(f"   ❌ Unexpected response format: {type(response_result)}")
                
        except Exception as e:
            print(f"   ❌ AI Agent error: {e}")
            import traceback
            print(f"     Error details: {traceback.format_exc()}")
    
    # Test 4: Direct Action Execution
    print("\n⚙️ Test 4: Direct Action Execution")
    try:
        from comprehensive_ai_agent import AgentAction
        import uuid
        
        test_action = AgentAction(
            action_id=str(uuid.uuid4()),
            action_type="get_inventory_by_location",
            description="Get inventory items for Clinical Laboratory",
            parameters={"location_name": "Clinical Laboratory", "include_details": True}
        )
        
        action_result = await agent._execute_action(test_action, "validation_test")
        
        if action_result and isinstance(action_result, dict) and "items" in action_result:
            validation_results["action_execution"] = True
            items = action_result["items"]
            print(f"   ✅ Action execution successful - {len(items)} items returned")
            if items:
                print(f"   📦 Sample item details: {items[0]}")
        else:
            print(f"   ❌ Action execution failed - unexpected result format")
            print(f"   📊 Result type: {type(action_result)}")
            if isinstance(action_result, dict):
                print(f"   📊 Result keys: {list(action_result.keys())}")
            
    except Exception as e:
        print(f"   ❌ Action execution error: {e}")
        import traceback
        print(f"     Error details: {traceback.format_exc()}")
    
    # Summary
    print("\n" + "=" * 60)
    print("🏆 VALIDATION SUMMARY")
    print("=" * 60)
    
    total_tests = len(validation_results)
    passed_tests = sum(validation_results.values())
    
    for test_name, passed in validation_results.items():
        status = "✅ PASS" if passed else "❌ FAIL"
        print(f"   {test_name.replace('_', ' ').title()}: {status}")
    
    print(f"\n🎯 Overall Result: {passed_tests}/{total_tests} tests passed")
    
    if passed_tests == total_tests:
        print("🎉 ALL SYSTEMS OPERATIONAL - Issue may be in web interface integration")
    elif passed_tests >= total_tests * 0.8:
        print("⚠️ MOSTLY FUNCTIONAL - Minor issues to resolve")
    else:
        print("🚨 CRITICAL ISSUES - Major problems need fixing")
    
    return validation_results

# Run comprehensive validation
validation_summary = await comprehensive_validation()

🏥 COMPREHENSIVE HOSPITAL INVENTORY SYSTEM VALIDATION

🔗 Test 1: Database Connection
   ✅ Database initialized successfully
   📊 Found 124 total inventory items

📍 Test 2: Location-Based Inventory Filtering
   ✅ Location filtering works - 4 items in Clinical Laboratory
   📦 Sample items: ['Blood Collection Tubes (100 pack)', 'Disinfectant Wipes (Pack of 100)', 'Hand Sanitizer 500ml']

🤖 Test 3: AI Agent Response Generation

   Query 1: 'what are the inventory items I have in Clinical Laboratory location?'
   ⚠️ Response unclear - manual review needed
   📝 Response length: 925 characters
   🎯 First 200 chars: Based on the inventory inquiry for the Clinical Laboratory location, here are the items available in that specific area:
1. Laboratory Supplies: 
- Reagents
- Test tubes
- Microscopes
- Pipettes
- Cen...
   ⚙️ Actions executed: 1
     - get_inventory_by_location

   Query 2: 'show me items in the Clinical Laboratory'
   ⚠️ Response unclear - manual review needed
   📝 Response length

In [21]:
# Web Interface vs Direct Testing Comparison
async def web_interface_debugging():
    """Debug differences between web interface and direct testing"""
    print("🌐 WEB INTERFACE vs DIRECT TESTING ANALYSIS")
    print("=" * 60)
    
    # Check for different code paths
    print("\n🔍 Checking for Multiple AI Agent Implementations:")
    
    # Check backend/api directory
    backend_api_path = os.path.join("..", "backend", "api")
    if os.path.exists(backend_api_path):
        print(f"   📁 Backend API directory found: {backend_api_path}")
        
        # Look for AI agent files
        for file in os.listdir(backend_api_path):
            if "agent" in file.lower() and file.endswith(".py"):
                print(f"   🤖 Found agent file: {file}")
                
                # Check if it imports different modules
                file_path = os.path.join(backend_api_path, file)
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        content = f.read()
                        
                    if "get_inventory_by_location" in content:
                        print(f"      ✅ Contains location-based query functionality")
                    else:
                        print(f"      ❌ Missing location-based query functionality")
                        
                    if "ComprehensiveAIAgent" in content:
                        print(f"      ✅ Uses ComprehensiveAIAgent class")
                    else:
                        print(f"      ❌ Uses different agent implementation")
                        
                except Exception as e:
                    print(f"      ⚠️ Could not read file: {e}")
    
    # Check for different database connections
    print("\n🗄️ Checking Database Integration Paths:")
    
    # Check if web interface uses different database config
    possible_configs = [
        os.path.join("..", "backend", "database", "config.py"),
        os.path.join("..", "backend", "api", "config.py"),
        os.path.join("..", "backend", "fixed_database_integration.py")
    ]
    
    for config_path in possible_configs:
        if os.path.exists(config_path):
            print(f"   📄 Found config: {os.path.basename(config_path)}")
            try:
                with open(config_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                    
                if "get_inventory_by_location" in content:
                    print(f"      ✅ Contains location filtering method")
                else:
                    print(f"      ❌ Missing location filtering method")
                    
            except Exception as e:
                print(f"      ⚠️ Could not read config: {e}")
    
    # Check for caching issues
    print("\n💾 Checking for Caching Issues:")
    
    # Look for cache directories
    cache_dirs = ["__pycache__", ".cache", "node_modules/.cache"]
    for cache_dir in cache_dirs:
        cache_path = os.path.join("..", cache_dir)
        if os.path.exists(cache_path):
            print(f"   📁 Found cache directory: {cache_dir}")
            # Get modification time
            try:
                import time
                mod_time = os.path.getmtime(cache_path)
                mod_time_str = time.ctime(mod_time)
                print(f"      🕒 Last modified: {mod_time_str}")
            except:
                pass
    
    # Check for environment differences
    print("\n🌍 Environment Analysis:")
    print(f"   🐍 Python version: {sys.version}")
    print(f"   📂 Current working directory: {os.getcwd()}")
    print(f"   📋 Python path: {sys.path[:3]}...")  # Show first 3 entries
    
    # Check for different requirements/dependencies
    requirements_files = [
        os.path.join("..", "backend", "api", "requirements.txt"),
        os.path.join("..", "ai_ml", "requirements.txt"),
        os.path.join("..", "requirements.txt")
    ]
    
    print("\n📋 Dependency Analysis:")
    for req_file in requirements_files:
        if os.path.exists(req_file):
            print(f"   📄 Found requirements: {os.path.basename(req_file)}")
            try:
                with open(req_file, 'r', encoding='utf-8') as f:
                    content = f.read()
                    lines = [line.strip() for line in content.split('\n') if line.strip() and not line.startswith('#')]
                    print(f"      📦 {len(lines)} packages listed")
                    
                    # Check for critical packages
                    critical_packages = ['openai', 'psycopg2', 'fastapi', 'chromadb', 'langchain']
                    for package in critical_packages:
                        if any(package in line.lower() for line in lines):
                            print(f"      ✅ {package} dependency found")
                        else:
                            print(f"      ❌ {package} dependency missing")
                            
            except Exception as e:
                print(f"      ⚠️ Could not read requirements: {e}")
    
    # Recommendations
    print("\n" + "=" * 60)
    print("🎯 TROUBLESHOOTING RECOMMENDATIONS")
    print("=" * 60)
    
    recommendations = [
        "1. Check if web interface is using backend/api/enhanced_supply_agent.py instead of ai_ml/comprehensive_ai_agent.py",
        "2. Verify that backend API server is importing the updated database integration with location filtering",
        "3. Clear all Python cache files (__pycache__) and restart the backend server",
        "4. Ensure web interface is calling the correct API endpoint with location parameters",
        "5. Check for environment variable differences between direct testing and web deployment",
        "6. Verify that the frontend is sending the location parameter correctly in API requests",
        "7. Check API logs for actual requests/responses when using web interface"
    ]
    
    for rec in recommendations:
        print(f"   {rec}")
    
    print(f"\n🔧 Next Steps:")
    print(f"   1. Run this diagnostic notebook to validate direct functionality")
    print(f"   2. Compare results with web interface behavior")
    print(f"   3. Check backend API logs during web interface usage")
    print(f"   4. Verify API endpoint routing and parameter handling")

# Run web interface debugging
await web_interface_debugging()

🌐 WEB INTERFACE vs DIRECT TESTING ANALYSIS

🔍 Checking for Multiple AI Agent Implementations:
   📁 Backend API directory found: ..\backend\api
   🤖 Found agent file: enhanced_supply_agent.py
      ❌ Missing location-based query functionality
      ❌ Uses different agent implementation

🗄️ Checking Database Integration Paths:
   📄 Found config: config.py
      ❌ Missing location filtering method
   📄 Found config: fixed_database_integration.py
      ✅ Contains location filtering method

💾 Checking for Caching Issues:

🌍 Environment Analysis:
   🐍 Python version: 3.10.7 (tags/v3.10.7:6cc6b13, Sep  5 2022, 14:08:36) [MSC v.1933 64 bit (AMD64)]
   📂 Current working directory: e:\Rise Ai\Updated\Hospital-Operations-Logistics-Agentic-Platform_\ai_ml
   📋 Python path: ['e:\\Rise Ai\\Updated\\ai_ml', 'e:\\Rise Ai\\Updated\\Hospital-Operations-Logistics-Agentic-Platform_\\backend', 'e:\\Rise Ai\\Updated\\Hospital-Operations-Logistics-Agentic-Platform_\\backend']...

📋 Dependency Analysis:
   📄 

## 🏁 Diagnostic Notebook Usage Instructions

### How to Use This Diagnostic Tool

1. **Run All Cells in Sequence**: Execute each cell from top to bottom to get a complete system analysis

2. **Key Tests to Watch**:
   - **Database Connectivity**: Should show 124+ inventory items across multiple locations
   - **Location Filtering**: Should return specific items for Clinical Laboratory (e.g., Blood Collection Tubes, Reagent Test Kits)
   - **AI Agent Response**: Should generate specific, database-driven responses instead of generic text
   - **Action Execution**: Should successfully execute get_inventory_by_location actions

3. **Expected Results**:
   - ✅ All database tests should pass
   - ✅ AI agent should return real inventory items
   - ✅ Actions should execute without errors
   - ✅ Responses should be specific, not generic

### Interpreting Results

**If Direct Testing Works But Web Interface Doesn't**:
- The web interface may be using different code (backend/api vs ai_ml)
- Check API routing and parameter handling
- Verify frontend is sending location parameters correctly
- Clear cache and restart backend server

**If Direct Testing Also Fails**:
- Database connection issues
- Missing dependencies
- Environment configuration problems
- Code integration errors

### Next Steps After Running Diagnostics

1. **Document Results**: Note which tests pass/fail
2. **Compare with Web Interface**: Test the same queries in the web interface
3. **Check Logs**: Review backend API logs for differences
4. **Update Code**: Apply fixes based on diagnostic findings

### Quick Fix Commands (if needed)

```bash
# Clear Python cache
find . -name "__pycache__" -type d -exec rm -rf {} +

# Restart backend server
cd backend/api && python professional_main_smart.py

# Reinstall dependencies
pip install -r requirements.txt
```

---

**🎯 Purpose**: This notebook validates that the core hospital inventory system with location-based AI queries is working correctly in direct testing, helping identify if issues are in the web interface integration layer.

## 🚀 SOLUTION: Fix Web Interface Integration

### 🎯 Problem Identified:
The web interface debugging revealed that:
1. **✅ Direct Testing**: Uses `ai_ml/comprehensive_ai_agent.py` (enhanced with location queries)
2. **❌ Web Interface**: May have import issues or path problems preventing the enhanced agent from loading

### 🔧 Solution Steps:

#### Step 1: Verify AI Agent Import Path
The backend API tries to import from `ai_ml/comprehensive_ai_agent.py` but there might be a path issue.

#### Step 2: Ensure Enhanced Agent is Available
The web interface needs to successfully import and use the enhanced `ComprehensiveAIAgent` class.

#### Step 3: Test Web Interface Integration
After fixing the import, test that the web interface returns specific inventory data.

In [22]:
# Solution Implementation: Fix Web Interface AI Agent Integration
async def fix_web_interface_integration():
    """
    Fix the web interface to use the enhanced AI agent with location-based queries
    """
    print("🔧 FIXING WEB INTERFACE INTEGRATION")
    print("=" * 50)
    
    # Step 1: Test current import path from backend API perspective
    print("\n🔍 Step 1: Testing Backend API Import Path")
    
    import sys
    import os
    
    # Add the same path that the backend API uses
    backend_ai_path = os.path.join("..", "..", "ai_ml")
    absolute_ai_path = os.path.abspath(backend_ai_path)
    
    print(f"   📂 Backend AI path: {backend_ai_path}")
    print(f"   📂 Absolute AI path: {absolute_ai_path}")
    print(f"   ✅ Path exists: {os.path.exists(absolute_ai_path)}")
    
    # Test if we can import from this path
    if absolute_ai_path not in sys.path:
        sys.path.insert(0, absolute_ai_path)
        print(f"   ➕ Added to sys.path")
    
    try:
        # Try importing exactly like the backend API does
        from comprehensive_ai_agent import process_agent_conversation, get_comprehensive_agent
        print(f"   ✅ Successfully imported comprehensive_ai_agent functions")
        
        # Test the standalone function
        print(f"\n🧪 Step 2: Testing Standalone Function")
        test_response = await process_agent_conversation(
            user_message="what are the inventory items I have in Clinical Laboratory location?",
            user_id="web_test",
            session_id="integration_test"
        )
        
        if isinstance(test_response, dict) and "response" in test_response:
            response = test_response["response"]
            print(f"   ✅ Standalone function works!")
            print(f"   📝 Response length: {len(response)} characters")
            print(f"   🎯 Preview: {response[:150]}...")
            
            # Check if it contains specific data
            specific_indicators = ["Blood Collection Tubes", "ITEM-009", "Clinical Laboratory"]
            contains_specific = any(indicator in response for indicator in specific_indicators)
            
            if contains_specific:
                print(f"   🎯 ✅ Contains specific database data!")
                return True, process_agent_conversation
            else:
                print(f"   ⚠️ Still contains generic responses")
                return False, None
        else:
            print(f"   ❌ Unexpected response format: {type(test_response)}")
            return False, None
            
    except Exception as e:
        print(f"   ❌ Import failed: {e}")
        import traceback
        print(f"   🔍 Error details: {traceback.format_exc()}")
        return False, None

# Step 3: Create a simple API test
async def test_api_endpoint_simulation(process_func):
    """
    Simulate the API endpoint call to test if it would work
    """
    print(f"\n🌐 Step 3: Simulating API Endpoint Call")
    
    if not process_func:
        print(f"   ❌ No process function available")
        return False
    
    try:
        # Simulate the exact API call structure
        class MockRequest:
            def __init__(self, message, user_id="web_user", session_id=None):
                self.message = message
                self.user_id = user_id
                self.session_id = session_id
        
        request = MockRequest("what are the inventory items I have in Clinical Laboratory location?")
        
        # Process exactly like the API endpoint does
        agent_response = await process_func(
            user_message=request.message,
            user_id=request.user_id,
            session_id=request.session_id
        )
        
        # Format exactly like the API response
        api_response = {
            "response": agent_response.get("response", "I apologize, but I couldn't process your request properly."),
            "agent_status": "active",
            "session_id": agent_response.get("session_id"),
            "intent_analysis": agent_response.get("intent", {}),
            "actions_performed": agent_response.get("actions", []),
            "context_used": agent_response.get("context", []),
            "capabilities": agent_response.get("agent_capabilities", []),
            "timestamp": agent_response.get("timestamp"),
            "confidence": 0.95
        }
        
        print(f"   ✅ API simulation successful!")
        print(f"   📊 Response keys: {list(api_response.keys())}")
        print(f"   ⚙️ Actions performed: {len(api_response['actions_performed'])}")
        
        # Check response quality
        response_text = api_response["response"]
        specific_indicators = ["Blood Collection Tubes", "ITEM-009", "Clinical Laboratory", "34 pack"]
        contains_specific = any(indicator in response_text for indicator in specific_indicators)
        
        if contains_specific:
            print(f"   🎯 ✅ API would return specific data!")
            print(f"   📝 Sample response: {response_text[:200]}...")
            return True
        else:
            print(f"   ⚠️ API would still return generic responses")
            print(f"   📝 Sample response: {response_text[:200]}...")
            return False
            
    except Exception as e:
        print(f"   ❌ API simulation failed: {e}")
        import traceback
        print(f"   🔍 Error details: {traceback.format_exc()}")
        return False

# Run the fix
print("🚀 Starting Web Interface Integration Fix...")
import_success, process_func = await fix_web_interface_integration()
api_success = await test_api_endpoint_simulation(process_func)

print(f"\n" + "=" * 50)
print(f"🏆 INTEGRATION FIX RESULTS")
print(f"=" * 50)
print(f"   Import Test: {'✅ SUCCESS' if import_success else '❌ FAILED'}")
print(f"   API Test: {'✅ SUCCESS' if api_success else '❌ FAILED'}")

if import_success and api_success:
    print(f"\n🎉 WEB INTERFACE SHOULD NOW WORK!")
    print(f"   💡 If you still see generic responses:")
    print(f"     1. Restart the backend server: python professional_main_smart.py")
    print(f"     2. Clear browser cache (Ctrl+F5)")
    print(f"     3. Check server logs for import errors")
elif import_success:
    print(f"\n⚠️ Import works but API simulation has issues")
    print(f"   💡 The backend server may need to be restarted")
else:
    print(f"\n🚨 Import issues detected")
    print(f"   💡 The backend server may have path or dependency issues")

🚀 Starting Web Interface Integration Fix...
🔧 FIXING WEB INTERFACE INTEGRATION

🔍 Step 1: Testing Backend API Import Path
   📂 Backend AI path: ..\..\ai_ml
   📂 Absolute AI path: e:\Rise Ai\Updated\ai_ml
   ✅ Path exists: False
   ✅ Successfully imported comprehensive_ai_agent functions

🧪 Step 2: Testing Standalone Function
   ✅ Standalone function works!
   📝 Response length: 1430 characters
   🎯 Preview: Based on the inventory inquiry for the Clinical Laboratory location, here are the items available in that specific area:
1. Laboratory Supplies: This ...
   🎯 ✅ Contains specific database data!

🌐 Step 3: Simulating API Endpoint Call
   ✅ API simulation successful!
   📊 Response keys: ['response', 'agent_status', 'session_id', 'intent_analysis', 'actions_performed', 'context_used', 'capabilities', 'timestamp', 'confidence']
   ⚙️ Actions performed: 1
   🎯 ✅ API would return specific data!
   📝 Sample response: Based on the information provided, the inventory items available at the C